In [ ]:
%pylab inline

In [ ]:
import seaborn as sns

In [ ]:
%run ../src/model_test_lib.py

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
train = pd.read_pickle(r'..\data\processed\train_v1.pkl')
test = pd.read_pickle(r'..\data\processed\test_open_pred_v1.pkl')

In [ ]:
# train = train[(train['is_open']) == 1].reset_index(drop=True)
# test = test[(test['is_open']) == 1].reset_index(drop=True)

In [ ]:
train.head()

In [ ]:
len(train)

In [ ]:
features = train.columns.tolist()

In [ ]:
target_open = 'is_open'
target_click = 'is_click'
pid = 'id'
campaign_id = 'campaign_id'
user_id = 'user_id'

In [ ]:
features.remove(target_open)
features.remove(target_click)
features.remove(pid)
features.remove(campaign_id)
features.remove(user_id)

In [ ]:
memory_cleaner()

### Predict all is_open = mode(is_click)

In [ ]:
y = [int(train['is_click'].mode()[0])] * len(train)

In [ ]:
metrics.accuracy_score(train['is_click'], y)

In [ ]:
metrics.roc_auc_score(train['is_click'], y)

### Let's check how some basic models work without any hyperparameters

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model_names = ['LR', 'DTC']
model_list = [
    LogisticRegression(),
    DecisionTreeClassifier()
]

In [ ]:
test_classifiers(model_names, model_list, train, features, target_click)

### Let's tune the hyperparameter class_weight to be balanced, so that the algorithm treats them equal

In [ ]:
model_names = ['LR', 'DTC']
model_list = [
    LogisticRegression(class_weight='balanced'),
    DecisionTreeClassifier(class_weight='balanced')
]

In [ ]:
test_classifiers(model_names, model_list, train, features, target_click)

In [ ]:
original_features = ['no_of_images', 'no_of_internal_links', 'no_of_sections', 'total_links']
derived_feature_external_link = ['no_of_external_links']
time_features = ['day_of_week', 'time_group']
communication_features = [col for col in features if col.startswith('communication_')]
binned_features = [col for col in features if col.startswith('bin_')]
sub_features = [col for col in features if col.startswith('sub -')]
body_features = [col for col in features if col.startswith('body -')]

In [ ]:
# Test to check if newly grouped feature contains all the features
set(features) == set(original_features + derived_feature_external_link + time_features + communication_features 
                     + binned_features + sub_features + body_features)

#### Let's not include the original feature and the external link features as they are already part of binned features. And create a seprate group of features that has at least 0.05 event distinguishing capability

In [ ]:
new_feature_group = time_features + communication_features + binned_features + sub_features + body_features + [target_open]

In [ ]:
selected_features = []
for feat in new_feature_group:
    group = (train.groupby(feat)['is_click'].sum() / train.groupby(feat)['is_click'].count())
    
    if len(group) == 2:
        present = group.get_values()[1]
        absent = group.get_values()[0]
        
        if abs(present - absent) >= 0.05:
            selected_features.append(feat)

In [ ]:
selected_features

In [ ]:
# Retest the same balanced model
test_classifiers(model_names, model_list, train, selected_features, target_click)

In [ ]:
len(new_feature_group)

In [ ]:
selected_features = select_features(train, new_feature_group, target_open, k=100)

In [ ]:
test_classifiers_roc_auc_score(model_names, model_list, train, selected_features, target_click)

In [ ]:
model_names = ['LR', 'DTC']
model_list = [
    LogisticRegression(class_weight='balanced', max_iter=1000),
    DecisionTreeClassifier(class_weight='balanced', min_samples_split=0.05)
]
test_classifiers_roc_auc_score(model_names, model_list, train, new_feature_group, target_click)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(train[new_feature_group], train[target_click], test_size=0.3, random_state=42)

In [ ]:
model = DecisionTreeClassifier(class_weight='balanced', min_samples_split=0.05)

In [ ]:
model.fit(train_X, train_y)

In [ ]:
pred_proba = model.predict_proba(test_X)

In [ ]:
metrics.roc_auc_score(test_y, pred_proba[:,1])

### LGBM

In [ ]:
import lightgbm as lgb

In [ ]:
# set the data in format lgb accepts
# d_train = lgb.Dataset(train_X, label=train_y)
d_train = lgb.Dataset(train[new_feature_group], label=train[target_click])

In [ ]:
# set parameters
## you can tune the parameters can try to better score

params = {'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    'is_unbalance': True,
    'metric': 'auc',
    'learning_rate': 0.1, 
    'max_depth': 51, 
    'num_leaves': 175, 
    'feature_fraction': 0.5, 
    'max_bin': 256,
    'bagging_fraction': 0.8,
    'bagging_freq': 3
}

In [ ]:
lgb_cv = lgb.cv(params, d_train, num_boost_round=1000, nfold= 5, shuffle=True, stratified=True,
                verbose_eval=20, early_stopping_rounds=500)

In [ ]:
## get nround value which had lowest error
nround_max = lgb_cv['auc-mean'].index(np.max(lgb_cv['auc-mean']))
nround_min = lgb_cv['auc-mean'].index(np.min(lgb_cv['auc-mean']))
print("MAX Rounds = " + str(nround_max) + " & max auc: " + str(np.max(lgb_cv['auc-mean'])))
print("MIN Rounds = " + str(nround_min) + " & min auc: " + str(np.min(lgb_cv['auc-mean'])))

In [ ]:
model = lgb.train(params, d_train, num_boost_round=nround_max)

In [ ]:
import pickle

In [ ]:
filename = r'..\models\model_lgbm_click_v2.pkl'
# pickle.dump(model, open(filename, 'wb'))

In [ ]:
# Read and predict
model = pickle.load(open(filename, 'rb'))

In [ ]:
pred = model.predict(train[new_feature_group])

In [ ]:
pred[:5]

In [ ]:
pred_class = np.where(pred > 0.5, 1, 0)

In [ ]:
metrics.roc_auc_score(train['is_click'], pred_class)

In [ ]:
best_ac = -1
best_i = -1
for i in np.arange(0.350, 0.800, 0.001):
    # ac = metrics.accuracy_score(test_y, np.where(preds > i, 1, 0))
    ac = metrics.roc_auc_score(train[target_click], np.where(pred > i, 1, 0))
    
    if ac > best_ac:
        print('i = ' + str(i) + ', ac = ' + str(ac))
        best_ac = ac
        best_i = i

In [ ]:
pred = model.predict(test[new_feature_group])

In [ ]:
pred_class = np.where(pred > 0.5, 1, 0)

In [ ]:
pred_class.sum() / len(pred_class)

In [ ]:
test['is_click'] = pred

In [ ]:
test[[pid, target_click]].to_csv(r'..\data\processed\submission_lgbm_v2.csv', index=False)